## Import libraries

In [ ]:
from nba_api.stats.static import teams, players
from pandas import DataFrame
import pandas as pd

## Player Data in nba_api

In [ ]:
players_df = DataFrame(players.get_players())
players_df.sample(15)

In [ ]:
players_df.shape

### and teams

In [ ]:
team_df = DataFrame(teams.get_teams())
team_df

In [ ]:
type(players.get_players())

In [ ]:
type(teams.get_teams())

## Getting data with nba_api

### 1. Find an nba_api endpoint

### 2. Import nba_api Endpoint in Python

In [ ]:
from nba_api.stats.endpoints import commonplayerinfo

In [ ]:
# Example usage
player_id = 2544  # LeBron James
player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
player_info_df = player_info.get_data_frames()[0]

player_info_df

In [ ]:
player_info_df.columns

In [ ]:
from nba_api.stats.endpoints.teamyearbyyearstats import TeamYearByYearStats

In [ ]:
# TeamYearByYearStats?

### 3. Calling the Endpoint

In [ ]:
celtics_id = '1610612738'
celtics_data = TeamYearByYearStats(celtics_id)
celtics_data

### 4. Getting Data in Pandas

In [ ]:
celtics_df = celtics_data.get_data_frames()[0]
celtics_df

## Play by Play Example

### 1. Find an nba_api endpoint

### 2. Import nba_api Endpoint in Python

In [ ]:
from nba_api.stats.endpoints.playbyplayv2 import PlayByPlayV2

In [ ]:
#PlayByPlayV2?

### 3. Calling the EndPoint

In [ ]:
game_id = '0021700807'
pbp_data = PlayByPlayV2(game_id)
pbp_data

### 4. Getting Data in Pandas

In [ ]:
pbp_df = pbp_data.get_data_frames()[0]
pbp_df.head()

In [ ]:
pbp_df[['HOMEDESCRIPTION', 'VISITORDESCRIPTION']].head(10)

### for mutiple games

In [ ]:
from nba_api.stats.endpoints.teamgamelog import TeamGameLog

In [ ]:
# TeamGameLog?

In [ ]:
team_df.query("abbreviation == 'GSW'")[['id', 'full_name']]

In [ ]:
warriors_id = '1610612744'

In [ ]:
gsw_log = TeamGameLog(warriors_id).get_data_frames()[0]
gsw_log.head()

In [ ]:
gsw_game_ids = list(gsw_log['Game_ID'].head())
gsw_game_ids

In [ ]:
gsw_pbp_df1 = pd.concat([
    PlayByPlayV2(x).get_data_frames()[0] for x in gsw_game_ids
], ignore_index=True)

gsw_pbp_df1[['HOMEDESCRIPTION', 'VISITORDESCRIPTION']].head(10)

In [ ]:
def get_pbp_data(game_id):
    try:
        pbp_df = PlayByPlayV2(game_id).get_data_frames()[0]
    except Exception as _:
        pbp_df = DataFrame()
        
    return pbp_df

In [ ]:
get_pbp_data('0022301155')[['HOMEDESCRIPTION', 'VISITORDESCRIPTION']].head(10)

In [ ]:
get_pbp_data('XXXXXXXXXX')

In [ ]:
gsw_game_ids = gsw_game_ids + ['XXXXXXXXXX']

In [ ]:
gsw_game_ids

In [ ]:
pbp_df_all = DataFrame()
bad_game_ids = []
for gid in gsw_game_ids:
    print(gid)
    try:
        working_df = PlayByPlayV2(gid).get_data_frames()[0]
    except Exception as _:
        bad_game_ids = bad_game_ids + [gid]
        continue
        
    pbp_df_all = pd.concat([pbp_df_all, working_df], ignore_index=True)

In [ ]:
pbp_df_all['GAME_ID'].value_counts()

In [ ]:
bad_game_ids

In [ ]:
import sqlite3
from os import path

In [ ]:
# handle directories
DATA_DIR = r'C:\Users\Maintenant pret\OneDrive\Documents\Learn to Code With Basketball\code-basketball-files-main\data'

# create connection
conn = sqlite3.connect(path.join(DATA_DIR, 'basketball-data.sqlite'))

In [ ]:
# write it to sql
players_df.to_sql('player', conn, index=False, if_exists='replace')
celtics_df.to_sql('celtics', conn, index=False, if_exists='replace')
gsw_pbp_df1.to_sql('gsw_pbp', conn, index=False, if_exists='replace')
pbp_df_all.to_sql('pbp_all', conn, index=False, if_exists='replace')

In [ ]:
df = pd.read_sql(
    """
    SELECT HOMEDESCRIPTION, VISITORDESCRIPTION
    FROM pbp_all
    """, conn)

df.head()